In [24]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import os
import io
import math
import json
import random
import ndjson
import pandas
import cPickle as pkl
import numpy as np
import tensorflow as tf
from PIL import Image, ImageDraw

In [27]:
from tqdm import tqdm_notebook as tqdm

In [31]:
_tmp = pandas.read_csv('../../data_all/bee.csv')
strokes = json.loads(_tmp.iloc[20000,1])

image = Image.new("P", (256,256), color=255)
image_draw = ImageDraw.Draw(image)
for stroke in strokes:
    for i in range(len(stroke[0])-1):
        image_draw.line([stroke[0][i], 
                         stroke[1][i],
                         stroke[0][i+1], 
                         stroke[1][i+1]],
                        fill=0, width=5)
image = image.resize((128,128))

image.save('x.png')

In [36]:
print strokes[0]

[[148, 133, 121, 42, 29, 21, 21, 26, 36, 52, 81, 182, 203, 210, 214, 217, 209, 183, 164, 137], [74, 71, 73, 105, 117, 132, 162, 176, 190, 202, 215, 217, 204, 190, 169, 101, 94, 82, 77, 75]]


In [12]:
def class_name(p):
    return p.split('/')[-1].split('.')[0]

data_root = '../../data_all/'
data_files = [os.path.join(data_root, x) for x in os.listdir(data_root)]
class_id_map = {}
for i, x in enumerate(data_files):
    class_id_map[class_name(x)] = i

def strokes_to_image_str(strokes):
    image = Image.new("P", (256,256), color=255)
    image_draw = ImageDraw.Draw(image)
    for stroke in strokes:
        for i in range(len(stroke[0])-1):
            image_draw.line([stroke[0][i], 
                             stroke[1][i],
                             stroke[0][i+1], 
                             stroke[1][i+1]],
                             fill=0, width=5)
    img_size = 128
    image = image.resize((img_size,img_size))

    with io.BytesIO() as output:
        image.save(output, format="PNG")
        contents = output.getvalue()
    return contents

In [ ]:
"""
pkl_data = {}
for d in data_files:
    cn = class_name(d)
    data = pandas.read_csv(d)
    pkl_data[cn] = data

pkl.dump(pkl_data, file('pkled_all.pkl', 'wb'), 2)
"""

In [4]:
data_all = pkl.load(file('pkled_all.pkl', 'rb'))

In [32]:
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


tfr_root = '../../tfrecords_x4/'

tot_num_train = 0
tot_num_val = 0

all_samples = []

for cn in tqdm(data_all.keys()):
    df = data_all[cn]
    n_samples = len(df)
    
    cid = class_id_map[cn]
    
    tfr_val = os.path.join(tfr_root, 'val-%04d.tfrecord' % cid)
    
    n_samples_val = 1000
    n_samples_train = 40000
    
    #n_samples_train = n_samples - 1000
    
    tot_num_train += n_samples_train
    tot_num_val += n_samples_val
    
    
    for i in range(n_samples_train):
        img_raw = strokes_to_image_str(json.loads(df.iloc[i,1]))
    
        example = tf.train.Example(features=tf.train.Features(feature={
            'label': _int64_feature(cid),
            'image_raw': _bytes_feature(img_raw)}))
        
        all_samples.append(example.SerializeToString())
    
    
    writer_val = tf.python_io.TFRecordWriter(tfr_val)
    for i in range(n_samples - n_samples_val, n_samples):
        img_raw = strokes_to_image_str(json.loads(df.iloc[i,1]))
    
        example = tf.train.Example(features=tf.train.Features(feature={
            'label': _int64_feature(cid),
            'image_raw': _bytes_feature(img_raw)}))
        writer_val.write(example.SerializeToString())
        
    writer_val.close()
    
    
sample_per_shard = len(all_samples) / 100
if len(all_samples) % 100 != 0:
    sample_per_shard += 1

import random
random.shuffle(all_samples)
    
for i in range(100):
    tfr_train = os.path.join(tfr_root, 'train-%04d.tfrecord' % i)
    writer_train = tf.python_io.TFRecordWriter(tfr_train)
    
    for j in range(sample_per_shard*i, min(sample_per_shard*(i+1), len(all_samples))):
        writer_train.write(all_samples[j])
    writer_train.close()

In [19]:
print tot_num_train, tot_num_val, len(class_id_map)

3400000 340000 340


In [29]:
pkl.dump(class_id_map, file('class_id_map.pkl', 'wb'), 2)